In [ ]:
"""
import numpy as np
import pandas as pd

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
"""

/kaggle/input/humando/HumanDO.json


# Install Requirements

In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 47.2 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


# Import Libraries

In [ ]:
from datasets import load_dataset
import torch
from transformers import BertForMaskedLM, BertTokenizer, Trainer, TrainingArguments
from transformers import BertConfig
import pandas as pd
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import json
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from transformers import BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from tqdm import tqdm
import random
import time
import datetime
from sklearn.model_selection import GridSearchCV
from sklearn.base import BaseEstimator
from sklearn.metrics import classification_report
from sklearn.model_selection import RandomizedSearchCV
import random
import copy
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
import string
from sklearn.metrics import classification_report

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

# Load the Knowledge Graph

In [ ]:
json_file_path = '/content/HumanDO.json'

with open(json_file_path, "r") as file:
  hdo_data = json.load(file)

# Preprocess the Knowledge Graph

In [ ]:
disease_labels = []
text_sequences = []

#adding information such as definition, synonyms and xref into a text sequence
for graph in hdo_data["graphs"]:
    for node in graph.get("nodes", []):
        label = node.get("lbl", "No label available")
        meta_info = node.get("meta", {})
        definition = meta_info.get("definition", {}).get("val", "No definition available")
        synonyms = [syn["val"] for syn in meta_info.get("synonyms", [])]
        xrefs = [xref["val"] for xref in meta_info.get("xrefs", [])]

        #adding additional information from the knowledge graph
        relationships = []
        entities = []

        #combine all the information into a single text sequence
        text = f"{label}: {definition}. Synonyms: {', '.join(synonyms)}. Cross-references: {', '.join(xrefs)}. Relationships: {', '.join(relationships)}. Entities: {', '.join(entities)}"

        disease_labels.append(label)
        text_sequences.append(text)

# Convert into Dataframe

In [ ]:
hdo_data = pd.DataFrame({"text_sequence": text_sequences, "label": disease_labels})

In [ ]:
hdo_data.head()

,text_sequence,label
0,angiosarcoma: A vascular cancer that derives_f...,angiosarcoma
1,pterygium: A corneal disease that is character...,pterygium
2,disease of metabolism: A disease that involvin...,disease of metabolism
3,shrimp allergy: A crustacean allergy that has_...,shrimp allergy
4,aspirin allergy: A drug allergy that has_aller...,aspirin allergy


In [ ]:
len(hdo_data['label'].unique())

14081

In [ ]:
hdo_data

,text_sequence,label
0,angiosarcoma: A vascular cancer that derives_f...,angiosarcoma
1,pterygium: A corneal disease that is character...,pterygium
2,disease of metabolism: A disease that involvin...,disease of metabolism
3,shrimp allergy: A crustacean allergy that has_...,shrimp allergy
4,aspirin allergy: A drug allergy that has_aller...,aspirin allergy
...,...,...
14076,has broader match: A skos concept mapping used...,has broader match
14077,has close match: A skos concept mapping used t...,has close match
14078,has exact match: A skos concept mapping used t...,has exact match
14079,has narrower match: A skos concept mapping use...,has narrower match


# Grouping Classes

In [ ]:
class_list = [
    "dermatitis", "asthma", "metabolism", "fever", "aspergillosis",
    "tuberculosis", "deficiency", "infectious", "mucormycosis",
    "paracoccidioidomycosis", "tinea capitis", "mucormycosis",
    "diarrhea", "mycosis", "syndrome", "encephalitis", "hepatitis",
    "gastroenteritis", "capillariasis", "typhus", "exanthem",
    "neurosyphilis", "Charcot-Marie-Tooth", "deafness", "aciduria",
    "carcinoma in situ", "cancer", "amyloidosis", "tumor", "allergy", "fasciitis", "meningitis",
    "cough", "pneumonia", "keratitis", "urethritis", "lymphangitis", "impetigo", "hypoplasia",
    "epilepsy", "dysplasia", "ptosis", "failure", "hypothyroidism", "laxa", "glycosylation",
    "dystrophy", "abnormalities", "anomalies", "dystonia", "leukemia", "rickets",
    "ependymoma", "lymphoma", "myopathy", "granulomatosis", "neuronopathy", "seizures", "pigmentosa",
    "cardiomyopathy", "dyskinesia", "deafness", "lipofuscinosis", "paraplegia", "blindness",
    "holoprosencephaly", "anemia", "migraine", "ataxia", "hyperplasia", "chondrodysplasia", "yaws",
    "disorder", "endometriosis", "keratitis", "obsolete", "carcinoma", "pyrexia", "glomerulonephritis",
    "glioma", "astrocytoma", "adenocarcinoma", "leiomyoma", "ophthalmoplegia", "liposarcoma",
    "teratoma", "papilloma", "arcinoma", "arcoma", "melanoma", "lymphoma", "conjunctivitis"
]

In [ ]:
def find_word_in_label(label, words_list):
    for word in words_list:
        if word in label:
            return word
    return 'other'  #If none of the words from the list are found, replace with 'other'

#Replace labels with corresponding words or 'other'
for i, label in enumerate(hdo_data['label']):
    #Check if the word from the word list exists within the label
    word_found = find_word_in_label(label, class_list)
    hdo_data.at[i, 'label'] = word_found

In [ ]:
#Unique Labels
len(hdo_data['label'].unique())

84

In [ ]:
hdo_data = hdo_data.iloc[:14046]

# Shuffle the Data

In [ ]:
hdo_data = hdo_data.sample(frac=1, random_state=42)

In [ ]:
hdo_data.reset_index(inplace=True)

In [ ]:
hdo_data.drop(columns='index', inplace=True)

In [ ]:
hdo_data

,text_sequence,label
0,primary ovarian insufficiency 14: A primary ov...,other
1,choreatic disease: A movement disease characte...,other
2,gallbladder leiomyosarcoma: A gallbladder sarc...,arcoma
3,autosomal recessive nonsyndromic deafness 1B: ...,deafness
4,obsolete mucinous bronchioloalveolar lung carc...,obsolete
...,...,...
14041,congenital muscular dystrophy-dystroglycanopat...,dystrophy
14042,pityriasis rosea: A dermatitis that is charact...,other
14043,optic atrophy 8: An optic atrophy characterize...,other
14044,endocardium disease: A heart disease that is l...,other


# Reduce the number of Rows for the label as "Other"

Maintain consistency in data

In [ ]:
count = hdo_data.groupby('label').size().get('other', 0)
print(count)

5169


In [ ]:
# Assuming hdo_data is your original DataFrame and 'label' is the name of the column you want to filter
word = 'other'
desired_rows = 1000  # Specify the number of rows you want to keep

# Filter the DataFrame to include only the rows where the word is present
word_rows = hdo_data[hdo_data['label'].str.contains(word)]

# Check if the number of rows containing the word is greater than the desired_rows
if len(word_rows) > desired_rows:
    # Randomly select desired_rows from word_rows
    random_sample = word_rows.sample(n=desired_rows, random_state=42)  # You can choose any random_state
else:
    # If the number of rows containing the word is less than or equal to desired_rows, use all of them
    random_sample = word_rows

# Filter the DataFrame to exclude the rows where the word is present beyond the desired count
non_word_rows = hdo_data[~(hdo_data['label'].str.contains(word)) | hdo_data.index.isin(random_sample.index)]

# Reset index if necessary
non_word_rows.reset_index(drop=True, inplace=True)

# Overwrite the original DataFrame with the modified DataFrame
hdo_data = non_word_rows

In [ ]:
word_count = (hdo_data['label'] == 'other').sum()
print("Number of rows with 'other':", word_count)

Number of rows with 'other': 1000


In [ ]:
len(hdo_data)

9877

# Additional Preprocessing on Text Sequence

In [ ]:
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):

    #lowercasing
    text = text.lower()

    #tokenization
    tokens = word_tokenize(text)

    #removing punctuations
    tokens = [token for token in tokens if token not in string.punctuation]

    #removing stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]

    preprocessed_text = ' '.join(tokens)

    return preprocessed_text

In [ ]:
hdo_data['text_sequence'] = hdo_data['text_sequence'].apply(preprocess_text)

<ipython-input-23-92d907bf94c8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hdo_data['text_sequence'] = hdo_data['text_sequence'].apply(preprocess_text)


In [ ]:
hdo_data

,text_sequence,label
0,gallbladder leiomyosarcoma gallbladder sarcoma...,arcoma
1,autosomal recessive nonsyndromic deafness 1b a...,deafness
2,obsolete mucinous bronchioloalveolar lung carc...,obsolete
3,spinocerebellar ataxia type 8 autosomal domina...,ataxia
4,gallbladder small cell carcinoma definition av...,carcinoma
...,...,...
9872,otopalatodigital syndrome type 1 otopalatodigi...,syndrome
9873,fallopian tube germ cell cancer fallopian tube...,cancer
9874,obsolete recurrent pediatric cerebellar astroc...,obsolete
9875,congenital muscular dystrophy-dystroglycanopat...,dystrophy


# Tokenize the Sequences

In [ ]:
#load the BioBERT tokenizer and apply tokeinzation on text_sequences
tokenizer = BertTokenizer.from_pretrained("monologg/biobert_v1.1_pubmed", do_lower_case=False)

tokenized_texts = [tokenizer.tokenize(text) for text in hdo_data["text_sequence"]]

max_seq_length = 512
tokenized_texts = [text[:max_seq_length] for text in tokenized_texts]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/51.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/433 [00:00<?, ?B/s]

# Apply Padding and Attention masks

In [ ]:
padded_sequences = [text + ['[PAD]'] * (max_seq_length - len(text)) for text in tokenized_texts]

#converting tokenized sequences to input IDs
input_ids = [tokenizer.convert_tokens_to_ids(text) for text in padded_sequences]

#creating attention masks
attention_masks = [[1 if token != '[PAD]' else 0 for token in text] for text in padded_sequences]

In [ ]:
input_ids = torch.tensor(input_ids)
attention_masks = torch.tensor(attention_masks)

#encode labels
label_encoder = LabelEncoder()
hdo_data["label_encoded"] = label_encoder.fit_transform(hdo_data["label"])
labels = torch.tensor(hdo_data["label_encoded"])

<ipython-input-27-83e33de1ea83>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hdo_data["label_encoded"] = label_encoder.fit_transform(hdo_data["label"])


# Truncate the Text

In [ ]:
#truncating tokenized text sequences which exceed the maximum length
exceeding_indices = [i for i, text in enumerate(tokenized_texts) if len(text) > max_seq_length]
for i in exceeding_indices:
    input_ids[i] = input_ids[i][:max_seq_length]
    attention_masks[i] = attention_masks[i][:max_seq_length]

# Initialize the BioBERT Model

In [ ]:
config = BertConfig.from_pretrained("monologg/biobert_v1.1_pubmed", num_labels=len(set(hdo_data["label_encoded"])))

model = BertForSequenceClassification.from_pretrained("monologg/biobert_v1.1_pubmed", config=config)

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/biobert_v1.1_pubmed and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Split the data into Train-Test (85:15)

In [ ]:
len(labels)

9877

In [ ]:
batch_size = 16

train_inputs, val_inputs, train_labels, val_labels = train_test_split(input_ids, labels, random_state=42, test_size=0.15)
train_masks, val_masks, _, _ = train_test_split(attention_masks, input_ids, random_state=42, test_size=0.15)

# Create Dataloaders

In [ ]:
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

val_data = TensorDataset(val_inputs, val_masks, val_labels)
val_dataloader = DataLoader(val_data, batch_size=batch_size)

**Set Parameters**

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.train()
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)

# Train BioBERT

In [ ]:
num_epochs = 10
for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}/{num_epochs}")
    print("-" * 10)

    #training loop
    model.train()
    train_loss = 0

    for batch in tqdm(train_dataloader, desc="Training"):
        batch = tuple(t.to(device) for t in batch)
        inputs = {"input_ids": batch[0], "attention_mask": batch[1], "labels": batch[2]}
        optimizer.zero_grad()
        outputs = model(**inputs)
        loss = outputs.loss
        train_loss += loss.item()
        loss.backward()
        optimizer.step()
    avg_train_loss = train_loss / len(train_dataloader)
    print(f"Average training loss: {avg_train_loss:.4f}")

Epoch 1/10
----------


Training: 100%|██████████| 525/525 [12:26<00:00,  1.42s/it]


Average training loss: 1.9956
Epoch 2/10
----------


Training: 100%|██████████| 525/525 [12:32<00:00,  1.43s/it]


Average training loss: 0.5722
Epoch 3/10
----------


...

Average training loss: 0.0536
Epoch 9/10
----------


Training: 100%|██████████| 525/525 [12:31<00:00,  1.43s/it]


Average training loss: 0.0539
Epoch 10/10
----------


Training: 100%|██████████| 525/525 [12:31<00:00,  1.43s/it]

Average training loss: 0.0390


# Testing of BioBERT

In [ ]:
model.eval()
predictions = []
true_labels = []

with torch.no_grad():
    for batch in tqdm(val_dataloader, desc="Evaluation"):
        batch = tuple(t.to(device) for t in batch)
        inputs = {"input_ids": batch[0], "attention_mask": batch[1]}
        labels = batch[2].cpu().numpy()
        outputs = model(**inputs)
        logits = outputs.logits
        predicted_labels = torch.argmax(logits, dim=1).cpu().numpy()
        predictions.extend(predicted_labels)
        true_labels.extend(labels)

print("Classification Report:")
print(classification_report(true_labels, predictions, zero_division="warn"))

Evaluation: 100%|██████████| 93/93 [00:50<00:00,  1.83it/s]

Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.93      0.97        15
           1       1.00      1.00      1.00         2
           2       1.00      1.00      1.00         2
           3       1.00      1.00      1.00        14
           4       0.67      1.00      0.80         2
           5       1.00      1.00      1.00         8
           6       0.50      1.00      0.67         1
           7       0.98      1.00      0.99        40
           9       1.00      1.00      1.00         9
          10       1.00      1.00      1.00         4
          11       0.95      1.00      0.98        20
          12       1.00      1.00      1.00         3
          13       1.00      0.98      0.99        61
          14       0.00      0.00      0.00         1
          15       1.00      1.00      1.00        93
          16       1.00      1.00      1.00         6
          17       1.00      1.00      1.00         5
    


/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Initialize the BERT model and tokenizer

In [ ]:
# Load BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
tokenized_texts = [tokenizer.tokenize(text) for text in hdo_data["text_sequence"]]

max_seq_length = 512
tokenized_texts = [text[:max_seq_length] for text in tokenized_texts]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
padded_sequences = [text + ['[PAD]'] * (max_seq_length - len(text)) for text in tokenized_texts]

# Convert tokenized sequences to input IDs
input_ids = [tokenizer.convert_tokens_to_ids(text) for text in padded_sequences]

# Create attention masks
attention_masks = [[1 if token != '[PAD]' else 0 for token in text] for text in padded_sequences]

In [ ]:
input_ids = torch.tensor(input_ids)
attention_masks = torch.tensor(attention_masks)

# Encode labels
label_encoder = LabelEncoder()
hdo_data["label_encoded"] = label_encoder.fit_transform(hdo_data["label"])
labels = torch.tensor(hdo_data["label_encoded"])

<ipython-input-33-9557e5c3cc3c>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hdo_data["label_encoded"] = label_encoder.fit_transform(hdo_data["label"])


In [ ]:
# Handle sequence length error
# If any tokenized sequence exceeds the maximum length, truncate it
exceeding_indices = [i for i, text in enumerate(tokenized_texts) if len(text) > max_seq_length]
for i in exceeding_indices:
    input_ids[i] = input_ids[i][:max_seq_length]
    attention_masks[i] = attention_masks[i][:max_seq_length]

In [ ]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(set(hdo_data["label_encoded"])))
len(set(hdo_data["label_encoded"]))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


84

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.train()
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)

In [ ]:
num_epochs = 10
for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}/{num_epochs}")
    print("-" * 10)

    # Training loop
    model.train()
    train_loss = 0
    for batch in tqdm(train_dataloader, desc="Training"):
        batch = tuple(t.to(device) for t in batch)
        inputs = {"input_ids": batch[0], "attention_mask": batch[1], "labels": batch[2]}
        optimizer.zero_grad()
        outputs = model(**inputs)
        loss = outputs.loss
        train_loss += loss.item()
        loss.backward()
        optimizer.step()
    avg_train_loss = train_loss / len(train_dataloader)
    print(f"Average training loss: {avg_train_loss:.4f}")

Epoch 1/10
----------


Training: 100%|██████████| 525/525 [12:29<00:00,  1.43s/it]


Average training loss: 2.3916
Epoch 2/10
----------


Training: 100%|██████████| 525/525 [12:37<00:00,  1.44s/it]


Average training loss: 0.9231
Epoch 3/10
----------


...

Average training loss: 0.0856
Epoch 9/10
----------


Training: 100%|██████████| 525/525 [12:36<00:00,  1.44s/it]


Average training loss: 0.0683
Epoch 10/10
----------


Training: 100%|██████████| 525/525 [12:36<00:00,  1.44s/it]

Average training loss: 0.0609


In [ ]:
model.eval()
predictions = []
true_labels = []

with torch.no_grad():
    for batch in tqdm(val_dataloader, desc="Evaluation"):
        batch = tuple(t.to(device) for t in batch)
        inputs = {"input_ids": batch[0], "attention_mask": batch[1]}
        labels = batch[2].cpu().numpy()
        outputs = model(**inputs)
        logits = outputs.logits
        predicted_labels = torch.argmax(logits, dim=1).cpu().numpy()
        predictions.extend(predicted_labels)
        true_labels.extend(labels)

print("Classification Report:")
print(classification_report(true_labels, predictions, zero_division="warn"))

Evaluation: 100%|██████████| 93/93 [00:52<00:00,  1.78it/s]

Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.93      0.97        15
           1       1.00      1.00      1.00         2
           2       1.00      1.00      1.00         2
           3       1.00      1.00      1.00        14
           4       1.00      1.00      1.00         2
           5       0.88      0.88      0.88         8
           6       1.00      1.00      1.00         1
           7       0.95      1.00      0.98        40
           9       1.00      1.00      1.00         9
          10       0.80      1.00      0.89         4
          11       0.95      0.95      0.95        20
          12       1.00      1.00      1.00         3
          13       1.00      0.95      0.97        61
          14       0.00      0.00      0.00         1
          15       0.98      1.00      0.99        93
          16       1.00      1.00      1.00         6
          17       1.00      1.00      1.00         5
    


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
